## Silverbox

Silverbox is the name of a nonlinear system identification benchmark, proposed in 2004. Data, baselines and more info can be found at http://nonlinearbenchmark.org/#Silverbox.

State-space formulation of Silverbox's dynamics:

$$\begin{align}
\mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + \kappa(x(t)) x(t) =&\ u(t) + w(t) \\
\kappa(x(t)) =&\ \alpha + \beta x^2(t) \\
y(t) =&\ x(t) + e(t)
\end{align}$$

where
$$\begin{align}
\mu     =&\ \text{mass} \\
\nu     =&\ \text{viscous damping} \\
\kappa(x(t)) =&\ \text{nonlinear spring} \\
y(t)    =&\ \text{observation (displacement)} \\
x(t)    =&\ \text{state (displacement)} \\
u(t)    =&\ \text{force} \\
e(t)    =&\ \text{measurement noise} \\
w(t)    =&\ \text{process noise}
\end{align}$$

### Steps to solve

I now take a series of steps to re-write this problem:

#### 1. Assume constant spring coefficient κ

$$ \mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + κ x(t) = u(t) + w(t)$$

#### 2. Divide by leading coefficient

$$ \frac{d^2 x(t)}{dt^2} + \frac{\nu}{\mu} \frac{d x(t)}{dt} + \frac{κ}{\mu} x(t) = \frac{1}{\mu} u(t) + \frac{1}{\mu} w(t)$$

#### 3. Substitute standard variables

$$ \frac{d^2 x(t)}{dt^2} + 2\zeta \omega_0 \frac{d x(t)}{dt} + \omega_0^2 x(t) - \frac{u(t)}{\mu} = \frac{w(t)}{\mu}$$

where $$\begin{align} 
\zeta =&\ \frac{\nu}{2\sqrt{\mu \kappa}} \\ 
\omega_0 =&\ \sqrt{\frac{\kappa}{\mu}} \, .
\end{align}$$

#### 4. Apply Euler's method to obtain difference equation (step size is 1)

-> Forward Euler:

$$\begin{align}
\frac{x(t+2h)-2x(t+h)+x(t)}{h^2} + 2\zeta \omega_0 \frac{x(t+h)-x(t)}{h} + \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t+2) + 2(\zeta \omega_0 - 1) x(t+1) + (1 - 2 \zeta \omega_0 + \omega_0^2) x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} 
\end{align}$$

-> Backward Euler:

$$\begin{align}
\frac{x(t)-2x(t-h)+x(t-2h)}{h^2} - 2\zeta \omega_0 \frac{x(t)-x(t-h)}{h} - \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu}  \\
(1 + 2 \zeta \omega_0 + \omega_0^2)x(t) - 2(1 + \zeta \omega_0)x(t-1) - x(t-2) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t) - \frac{2(1 + \zeta \omega_0)}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-1) - \frac{1}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-2) - \frac{u(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)} =&\ \frac{w(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)}
\end{align}$$
    
Change to shorthand notation:

$$x_t - \alpha x_{t-1} - \beta x_{t-2} - \gamma u_t = \gamma w_t$$

where 
$$\begin{align} 
\alpha =&\ \frac{2(1 + \zeta \omega_0)}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\beta =&\ \frac{1}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\gamma =&\ \frac{1}{\mu(1 + 2 \zeta \omega_0 + \omega_0^2)}
\end{align}$$

#### 5. Convert to multivariate first-order difference form

Stick to backward Euler (matches AR structure)
- Backward Euler:

    $$z_t = M z_{t-1} + N u_t + N w_t$$

    where $z_t = [x_t\ \ x_{t-1}]$, $M = [α \ \ β; 1\ \ 0]$, $N = [γ\ \ 0]$

#### 6. Convert to Gaussian probability

- Backward Euler:

$$z_t \sim \mathcal{N}(M z_{t-1} + N u_t, N \tau)$$

where $w_t \sim \mathcal{N}(0, \tau)$

#### 7. Observation likelihood

$$y_t \sim \mathcal{N}(c z_t, σ)$$

where $e_t \sim \mathcal{N}(0, \sigma)$, $c = [1\ \ 0]$

Now, I need priors for $\alpha$, $\beta$, $\gamma$, $\tau$, $\sigma$. Given three equations and three unknowns, I can recover $\zeta$, $\omega_0$ and $\mu$ from $\alpha$, $\beta$, and $\gamma$. The variables are all strictly positive, which means they can be modeled by gamma distributions:

$$\begin{align}
\alpha \sim&\ Γ(1, 1e3) \\
\beta \sim&\ Γ(1, 1e3) \\
\gamma \sim&\ Γ(1, 1e3) \\
\tau \sim&\ Γ(1, 1e3) \\
\sigma \sim&\ Γ(1, 1e3) 
\end{align}$$

--> Implementation with ForneyLab and AR node

In [1]:
# Generate time-series
using CSV
using DataFrames
using Plots

# Read data from CSV file
df = CSV.read("../data/SNLS80mV.csv", ignoreemptylines=true)
df = select(df, [:V1, :V2])

# Shorthand
input = df[:,1]
output = df[:,2]

# Time horizon
T = size(df, 1)

131072

In [2]:
using ForneyLab
using LAR
using LAR.Node, LAR.Data
using ProgressMeter

┌ Info: Precompiling LAR [c3bc7fac-5998-4d64-8961-b7df36e0e4ce]
└ @ Base loading.jl:1273


I will introduce another shorthand: $\theta = [\alpha\ \ \beta]$.

In [3]:
# Start graph
g = FactorGraph()

# Static parameters
@RV log_θ ~ GaussianMeanPrecision(placeholder(:m_θ_t, dims=(2,)), placeholder(:w_θ_t, dims=(2,2)))
# @RV γ ~ Gamma(1, 1e3)
@RV τ ~ Gamma(placeholder(:a_τ_t, dims=(1,)), placeholder(:b_τ_t, dims=(1,)))

# Nonlinear function
f(x) = exp.(x)

# Nonlinear node
@RV θ ~ Nonlinear{Sampling}(log_θ, f, n_samples=100)

# I'm fixing measurement noise σ
σ = 10.

# Observation selection variable
c = [1, 0]

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z_t, dims=(2,)), placeholder(:w_z_t, dims=(2, 2)))

# Autoregressive node
@RV x_t ~ Autoregressive(θ, z_t, τ)

# Specify likelihood
@RV y_t ~ GaussianMeanPrecision(dot(c, x_t), σ)

# Placeholder for data
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(g)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 10066460828256618912 -->
 
 10066460828256618912 
 
 placeholder_w_θ_t 
 
<!-- 12658177492360399428 -->
 
 12658177492360399428 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 12658177492360399428--10066460828256618912 -->
 
 12658177492360399428--10066460828256618912 
 
 w_θ_t 
 1 out 
 3 w 
 
<!-- 16152576081149500078 -->
 
 16152576081149500078 
 
 placeholder_m_θ_t 
 
<!-- 12658177492360399428--16152576081149500078 -->
 
 12658177492360399428--16152576081149500078 
 
 m_θ_t 
 1 out 
 2 m 
 
<!-- 11561610975600872986 -->
 
 11561610975600872986 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 11561610975600872986--12658177492360399428 -->
 
 11561610975600872986--12658177492360399428 
 
 log_θ 
 1 out 
 2 in1 
 
<!-- 6461504922442617841 -->
 
 6461504922442617841 
 
 placeholder_y_t 
 
<!-- 12633386357487947498 -->
 
 12633386357487947498 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 6461504922442617841--12633386357487947498 -->
 
 6461504922442617841--12633386357487947498 
 
 y_t 
 1 out 
 1 out 
 
<!-- 13123372721753029675 -->
 
 13123372721753029675 
 
 AR 
 autoregressive_1 
 
<!-- 13123372721753029675--11561610975600872986 -->
 
 13123372721753029675--11561610975600872986 
 
 θ 
 1 out 
 3 θ 
 
<!-- 12141503565838334300 -->
 
 12141503565838334300 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 13123372721753029675--12141503565838334300 -->
 
 13123372721753029675--12141503565838334300 
 
 z_t 
 1 out 
 2 x 
 
<!-- 9252705691294177222 -->
 
 9252705691294177222 
 
 Gam 
 gamma_1 
 
<!-- 13123372721753029675--9252705691294177222 -->
 
 13123372721753029675--9252705691294177222 
 
 τ 
 1 out 
 4 γ 
 
<!-- 18139494918092071265 -->
 
 18139494918092071265 
 
 clamp_2 
 
<!-- 3669571025453248457 -->
 
 3669571025453248457 
 
 placeholder_m_z_t 
 
<!-- 12141503565838334300--3669571025453248457 -->
 
 12141503565838334300--3669571025453248457 
 
 m_z_t 
 1 out 
 2 m 
 
<!-- 2077892164557743082 -->
 
 2077892164557743082 
 
 placeholder_w_z_t 
 
<!-- 12141503565838334300--2077892164557743082 -->
 
 12141503565838334300--2077892164557743082 
 
 w_z_t 
 1 out 
 3 w 
 
<!-- 15298021328419664212 -->
 
 15298021328419664212 
 
 clamp_1 
 
<!-- 7996364658096885881 -->
 
 7996364658096885881 
 
 placeholder_a_τ_t 
 
<!-- 17516857700423173961 -->
 
 17516857700423173961 
 
 dot 
 dotproduct_1 
 
<!-- 17516857700423173961--13123372721753029675 -->
 
 17516857700423173961--13123372721753029675 
 
 x_t 
 1 y 
 2 in1 
 
<!-- 17516857700423173961--15298021328419664212 -->
 
 17516857700423173961--15298021328419664212 
 
 clamp_1 
 1 out 
 3 in2 
 
<!-- 9252705691294177222--7996364658096885881 -->
 
 9252705691294177222--7996364658096885881 
 
 a_τ_t 
 1 out 
 2 a 
 
<!-- 13638475711419621676 -->
 
 13638475711419621676 
 
 placeholder_b_τ_t 
 
<!-- 9252705691294177222--13638475711419621676 -->
 
 9252705691294177222--13638475711419621676 
 
 b_τ_t 
 1 out 
 3 b 
 
<!-- 12633386357487947498--18139494918092071265 -->
 
 12633386357487947498--18139494918092071265 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 12633386357487947498--17516857700423173961 -->
 
 12633386357487947498--17516857700423173961 
 
 variable_1 
 1 out 
 2 m

In [7]:
# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, θ, τ, ids=[:z, :x, :θ, :τ])
algo = variationalAlgorithm(q, free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
eval(Meta.parse(source_code));
println(source_code)

In [8]:
# Inference parameters
num_iterations = 10

# Initialize marginal distribution and observed data dictionaries
data = Dict()
marginals = Dict()

# Initialize arrays of parameterizations
params_x = (zeros(T+1,1), zeros(T+1,1))
params_z = (zeros(T+1,2), zeros(T+1,2,2))
params_θ = (zeros(T+1,2), zeros(T+1,2,2))
params_τ = (zeros(T+1,1), zeros(T+1,1))

# Start progress bar
p = Progress(T, 1, "At time ")

# Perform inference at each time-step
for t = 1:T

    # Update progress bar
    update!(p, t)

    # Initialize marginals
    marginals[:x_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][t,1], w=params_x[2][t,1])
    marginals[:z_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_z[1][t,:], w=params_z[2][t,:,:])
#     marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_m_θ[t,:], w=params_w_θ[t,:,:])
    marginals[:θ] = ProbabilityDistribution(Multivariate, SampleList; s=[0., 0.], w=[1. 0.;0. 1.], diff_ent=nothing)
    marginals[:τ] = ProbabilityDistribution(Univariate, Gamma, a=params_τ[1][t,1], b=params_τ[2][t,2])
    
    data = Dict(:y_t => output[t],
                :m_z_t => params_z[1][t,:],
                :w_z_t => params_z[2][t,:,:],
                :m_θ_t => params_θ[1][t,:],
                :w_θ_t => params_θ[2][t,:,:],
                :a_τ_t => params_τ[1][t,1],
                :b_τ_t => params_τ[2][t,1])

    # Iterate variational parameter updates
    for i = 1:num_iterations

        stepx!(data, marginals)
        stepz!(data, marginals)
        stepθ!(data, marginals)
        stepτ!(data, marginals)
    end

    # Store current parameterizations of marginals
    params_x[1][t+1,1] = unSafeMean(marginals[:x_t])
    params_x[2][t+1,1] = unSafePrecision(marginals[:x_t])
    params_z[1][t+1,:] = unSafeMean(marginals[:z_t])
    params_z[2][t+1,:,:] = unSafePrecision(marginals[:z_t])
    params_θ[1][t+1,:] = unSafeMean(marginals[:θ])
    params_θ[2][t+1,:,:] = unSafePrecision(marginals[:θ])
    params_τ[1][t+1,1] = marginals[:τ].params[:a]
    params_τ[2][t+1,1] = marginals[:τ].params[:b]

end

BoundsError: BoundsError: attempt to access 131073×1 Array{Float64,2} at index [1, 2]